In [2]:
# !pip install openpyxl
from openpyxl import load_workbook
from openpyxl.utils.cell import range_boundaries

In [1]:
# pip install numpy
# pip install pandas
import numpy as np
import pandas as pd

In [3]:
# loading workbook and accessing NRs as lists
a = load_workbook("Template/template 1.0.0.xlsx")

c:\Users\Andrea.Baschiera\OneDrive - EFRAG\Desktop\Data Migration Tool\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Andrea.Baschiera\OneDrive - EFRAG\Desktop\Data Migration Tool\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
list_names = a.defined_names.keys()
pyxl_NR = a.defined_names

res = []
for NR in list_names:
    res.append(pyxl_NR[NR].attr_text.split("!")[0].replace("'", ""))

res

In [133]:
df1_filtered = df1[df1["name_ranges"] == "TypeOfPollutantAxis"].iloc[0]
for i in range(len(df1_filtered["cell_values"])):
    df1_filtered["cell_values"][i] = [df1_filtered["cell_values"][i][0]] # taking only first subelement from each sublist


In [134]:
for j in range(df1_filtered["cell_shapes"][3]+1 - df1_filtered["cell_shapes"][1] - len(df1_filtered["cell_values"])):
    df1_filtered["cell_values"].append([None])

In [ ]:
df1_filtered["cell_values"]

In [184]:
filtered = df1[df1['name_ranges'].str.contains('hypercube', case=False, na=False)]

filtered["name_ranges"].values[0]

'BreakdownOfAnnualMassFlowOfRelevantMaterialsUsedByTheUndertakingHypercube'

In [175]:
env = a["Environmental Disclosures"]

for row in range(df1_filtered["cell_shapes"][1], 82):
    env.cell(row=row, column=df1_filtered["cell_shapes"][0]).value = "ciao come" #df1_filtered["cell_values"][row - df1_filtered["cell_shapes"][1]][0]

a.save("Template/debug1.xlsx")

In [4]:
# accessing NRs as lists
all_NRs = a.defined_names
list_NRs = list(a.defined_names.keys())

In [172]:
(4,219) in merged_loc["Environmental Disclosures"].values.tolist()

True

In [165]:
sheet_names = a.sheetnames
merged_loc = pd.DataFrame()

for sheet in sheet_names:
    merged = list(a[sheet].merged_cells.ranges)
    merged_list = []
    for i in range(len(merged)):
        str(merged[i])
        merged_list.append(range_boundaries(str(merged[i]))[:2])
    merged_loc = pd.concat([merged_loc, pd.DataFrame({sheet: merged_list})], axis=1)

In [ ]:
merged = list(a["Environmental Disclosures"].merged_cells.ranges)
merged
# for i in range(len(merged)):
#     print(str(merged[i]))

In [9]:
# Iterating over NRs and accessing their attributes, storing in dict
dict_of_cell_references = {}
for NR in list_NRs:
    dict_of_cell_references[NR] = all_NRs[NR].attr_text.split("!")[1]

In [7]:
def letter_to_number(col):
    col = col.upper()
    n = 0
    for c in col:
        if not ("A" <= c <= "Z"):
            raise ValueError(f"Invalid column letter: {col}")
        n = n * 26 + (ord(c) - ord("A") + 1)
    return n

# examples
letter_to_number("A")   # 1

1

In [ ]:
# Creating shapes of the cell references from ranges
dict_of_cell_shapes = {}
for i in dict_of_cell_references.keys():
    a = dict_of_cell_references[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    dict_of_cell_shapes[i] = a

dict_of_cell_shapes

In [ ]:
df = pd.DataFrame({
    "name_ranges": list_NRs,
    "cell_references": list(dict_of_cell_references.values()),
    "cell_shapes": list(dict_of_cell_shapes.values()),
})
df

In [ ]:
# condition for seeing whether cell reference is a formula
a["Social Disclosures"].cell(row=46, column=4).data_type == "f"  # True

In [4]:
with_data = load_workbook("with_data.xlsx")
empty = load_workbook("empty.xlsx")
NR_old = with_data.defined_names
list_NR_old = list(with_data.defined_names.keys())
NR_new = empty.defined_names
list_NR_new = list(empty.defined_names.keys())

In [8]:
references_old = []
for NR in NR_old:
    references_old.append(NR_old[NR].attr_text.split("!")[1])

shapes_old = []
for i in range(len(list_NR_old)):
    a = references_old[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    a = [int(ciao) for ciao in a]
    shapes_old.append(a)

df_old = pd.DataFrame({
    "name_ranges": list_NR_old,
    "cell_references": references_old,
    "cell_shapes": shapes_old,
})
df_old

,name_ranges,cell_references,cell_shapes
0,number,$D$3,"[4, 3]"
1,number2,$D$5,"[4, 5]"
2,stringa,$F$3:$F$6,"[6, 3, 6, 6]"


In [9]:
# Extracting cell values from old workbook based on cell references and shapes

sheet_old = with_data["Sheet1"]

cell_values = []

for i in range(len(df_old)):
    if len(df_old["cell_shapes"][i]) == 2:
        cell_values.append([sheet_old[df_old["cell_references"][i]].value])
    else:
        cell_values_temp = []
        for row in sheet_old.iter_rows( min_row=df_old["cell_shapes"][i][1],
                                        max_row=df_old["cell_shapes"][i][3],
                                        min_col=df_old["cell_shapes"][i][0],
                                        max_col=df_old["cell_shapes"][i][2] ):
            cell_values_temp.append([])
            for col in row:
                cell_values_temp[-1].append(col.value)
        cell_values.append(cell_values_temp)

df_old["cell_values"] = cell_values
df_old


,name_ranges,cell_references,cell_shapes,cell_values
0,number,$D$3,"[4, 3]",[=45+54]
1,number2,$D$5,"[4, 5]",[=number]
2,stringa,$F$3:$F$6,"[6, 3, 6, 6]","[[ciao], [come], [va], [ciao]]"


In [10]:
references_new = []
for NR in NR_new:
    references_new.append(NR_new[NR].attr_text.split("!")[1])

shapes_new = []
for i in range(len(list_NR_new)):
    a = references_new[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    a = [int(ciao) for ciao in a]
    shapes_new.append(a)

df_new = pd.DataFrame({
    "name_ranges": list_NR_new,
    "cell_references": references_new,
    "cell_shapes": shapes_new,
})
df_new

,name_ranges,cell_references,cell_shapes
0,number,$C$7,"[3, 7]"
1,number2,$O$7,"[15, 7]"
2,stringa,$H$8:$H$11,"[8, 8, 8, 11]"


In [11]:
# merging values from old to new (within dataframes)

df_old_tomerge = df_old.iloc[:,[0,3]]
df_new_withvalues = pd.merge(df_new, df_old_tomerge)
df_new_withvalues


,name_ranges,cell_references,cell_shapes,cell_values
0,number,$C$7,"[3, 7]",[=45+54]
1,number2,$O$7,"[15, 7]",[=number]
2,stringa,$H$8:$H$11,"[8, 8, 8, 11]","[[ciao], [come], [va], [ciao]]"


In [12]:
# paste values into new workbook

sheet_new = empty["Sheet1"]

for i in range(len(df_new_withvalues)):
    if len(df_new_withvalues["cell_shapes"][i]) == 2:
        sheet_new[df_new_withvalues["cell_references"][i]].value = df_new_withvalues["cell_values"][i][0]
    else:
        for row in range( df_new_withvalues["cell_shapes"][i][1],
                                df_new_withvalues["cell_shapes"][i][3] + 1 ):
            for col in range( df_new_withvalues["cell_shapes"][i][0],
                              df_new_withvalues["cell_shapes"][i][2] + 1 ):
                sheet_new.cell(row=row, column=col).value = df_new_withvalues["cell_values"][i][row - df_new_withvalues["cell_shapes"][i][1]][col - df_new_withvalues["cell_shapes"][i][0]]

empty.save("new_with_data.xlsx")

In [ ]:
# avoid shadowing built-in 'list' and use positional indexing to prevent KeyError from non-consecutive DataFrame index
values_list = []
for i in range(len(df)):
    cell_val = df["cell_values"].iloc[i]  # use iloc for positional access
    if isinstance(cell_val, (list, tuple)):
        values_list.append(cell_val[0] if len(cell_val) > 0 else None)
    else:
        values_list.append(cell_val)

len(values_list)

In [35]:
for i in range(len(df)):
    print(f"value accessed: {df['cell_values'].iloc[i][0]} and df value column: {df['cell_values'].iloc[i]}")

value accessed: ['Duomo Square'] and df value column: [['Duomo Square'], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None]]
value accessed: ['Alachlor', None, None, 567, None, None, None, None] and df value column: [['Alachlor', None, None, 567, None, None, None, None], ['Anthracene', None, None, None, None, None, None, 67], ['Carbon dioxide (CO2)', None, None, None, None, None, 567, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, No

TypeError: 'NoneType' object is not subscriptable

In [53]:
ciao = pd.DataFrame({
    "a": [1,2,3,4,5,6],
    "b": [7,8,9,None,11,12]
})
ciao.iloc[3]["b"]

np.float64(nan)

In [29]:
from openpyxl.utils.cell import range_boundaries

range_string = "A2:H5"

# Get the boundaries
range_boundaries(range_string)

(1, 2, 8, 5)

In [45]:
df1[df1["name_ranges"] == "TypeOfPollutantAxis"]["cell_values"]

173    [[Alachlor, None, None], [Anthracene, None, No...
Name: cell_values, dtype: object

In [9]:
from openpyxl.utils import absolute_coordinate

In [10]:
def coord(list_of_dicts):
    list_of_names = ["General Information", "Environmental Disclosures", "Social Disclosures", "Governance Disclosures", "Fuel Converter"]
    for dict in list_of_dicts:
        name = list_of_names[list_of_dicts.index(dict)]
        for key in dict.keys():
            for i in range(len(dict[key])):
                dict[key][i] = absolute_coordinate(dict[key][i])
                dict[key][i] = name + "!" + dict[key][i]
    return list_of_dicts

In [11]:
dict_missingNR_GI = {
    "1.0.0": ["D6:E8", "D10:E12", "E99", "H107", "C139", "E159"],
    "1.0.1": ["D6:E8", "D10:E12", "E99", "H107", "C139", "E159"],
    "1.1.0": ["D6:E8", "D10:E12", "E99", "H108", "C140", "E160"],
    "1.1.1": ["D6:E8", "D10:E12", "E440", "H449", "C556", "E576"]
}

dict_missing_ED = {
    "1.0.0": ["I59:K61","D30:F44", "G30:I44", "J30:J44", "D57", "G10", "J19", "G29", "G74", "G124", "D175", "G214", "E235"],
    "1.0.1": ["I59:K61","D30:F44", "G30:I44", "J30:J44", "D57", "G10", "J19", "G29", "G74", "G124", "D175", "G214", "E235"],
    "1.1.0": ["I59:K61","D30:F44", "G30:I44", "J30:J44", "D57", "G10", "J19", "G29", "G74", "G124", "D175", "G214", "E235"],
    "1.1.1": ["I59:K61","D30:F44", "G30:I44", "J30:J44", "D57", "G10", "J19", "G29", "G74", "G184", "D309", "G433", "E539"]
}

dict_missingNR_SD = {
    "1.0.0": ["D43:H45", "D50", "D57:H58", "D60", "D74:H75"],
    "1.0.1": ["D43:H45", "D50", "D57:H58", "D60", "D74:H75"],
    "1.1.0": ["D44:H46", "D51", "D58:H59", "D61", "D75:H76"],
    "1.1.1": ["D134:H136", "D141", "D148:H149", "D151", "D165:H166"]
}

dict_missingNR_GD = {
    "1.0.0": ["H5", "H10", "H22:H26", "H30:H32"],
    "1.0.1": ["H5", "H10", "H22:H26", "H30:H32"],
    "1.1.0": ["H5", "H10", "H22:H26", "H30:H32"],
    "1.1.1": ["H5", "H10", "H22:H26", "H30:H32"]
}

dict_missingNR_Fuel = {
    "1.0.0": ["B10", "L10", "P10", "T10", "X10", "AB10", "AF10", "AJ10", "AN10", "AR10", "B16", "L16", "P16", "T16", "X16", "AB16", "AF16", "AJ16", "AN16", "AR16", "B17", "L17", "P17", "T17", "X17", "AB17", "AF17", "AJ17", "AN17", "AR17"],
    "1.0.1": ["B10", "L10", "P10", "T10", "X10", "AB10", "AF10", "AJ10", "AN10", "AR10", "B16", "L16", "P16", "T16", "X16", "AB16", "AF16", "AJ16", "AN16", "AR16", "B17", "L17", "P17", "T17", "X17", "AB17", "AF17", "AJ17", "AN17", "AR17"],
    "1.1.0": ["A10", "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "B10", "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17", "C18", "C19"],
    "1.1.1": ["A10", "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "B10", "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17", "C18", "C19"]
}

updated = coord([dict_missingNR_GI, dict_missing_ED, dict_missingNR_SD, dict_missingNR_GD, dict_missingNR_Fuel])
df_missingNR_GI = pd.DataFrame(updated[0])
df_missing_ED = pd.DataFrame(updated[1])
df_missingNR_SD = pd.DataFrame(updated[2])
df_missingNR_GD = pd.DataFrame(updated[3])
df_missingNR_Fuel = pd.DataFrame(updated[4])


In [12]:
df_missingNR_SD.loc[5] = {
    "1.0.0": "Social Disclosures!None",
    "1.0.1": "Social Disclosures!None",
    "1.1.0": "Social Disclosures!$E$27",
    "1.1.1": "Social Disclosures!$E$27"
}

In [17]:
missingNR_df = pd.concat([df_missingNR_GI, df_missing_ED, df_missingNR_SD, df_missingNR_GD, df_missingNR_Fuel], ignore_index=True)
missingNR_df.to_pickle("pickles/missingNR_df.pkl")

In [18]:
df = pd.read_pickle("pickles/missingNR_df.pkl")

In [19]:
df

,1.0.0,1.0.1,1.1.0,1.1.1
0,General Information!$D$6:$E$8,General Information!$D$6:$E$8,General Information!$D$6:$E$8,General Information!$D$6:$E$8
1,General Information!$D$10:$E$12,General Information!$D$10:$E$12,General Information!$D$10:$E$12,General Information!$D$10:$E$12
2,General Information!$E$99,General Information!$E$99,General Information!$E$99,General Information!$E$440
3,General Information!$H$107,General Information!$H$107,General Information!$H$108,General Information!$H$449
4,General Information!$C$139,General Information!$C$139,General Information!$C$140,General Information!$C$556
5,General Information!$E$159,General Information!$E$159,General Information!$E$160,General Information!$E$576
6,Environmental Disclosures!$I$59:$K$61,Environmental Disclosures!$I$59:$K$61,Environmental Disclosures!$I$59:$K$61,Environmental Disclosures!$I$59:$K$61
7,Environmental Disclosures!$D$30:$F$44,Environmental Disclosures!$D$30:$F$44,Environmental Disclosures!$D$30:$F$44,Environmental Disclosures!$D$30:$F$44
8,Environmental Disclosures!$G$30:$I$44,Environmental Disclosures!$G$30:$I$44,Environmental Disclosures!$G$30:$I$44,Environmental Disclosures!$G$30:$I$44
9,Environmental Disclosures!$J$30:$J$44,Environmental Disclosures!$J$30:$J$44,Environmental Disclosures!$J$30:$J$44,Environmental Disclosures!$J$30:$J$44


In [ ]:
list_of_sheets = []
list_of_ranges = []

for i in range(len(missingNR_df)):
    list_of_sheets.append(missingNR_df.iloc[:,0].values[i].split("!")[0])
    list_of_ranges.append(missingNR_df.iloc[:,0].values[i].split("!")[1])

In [12]:
list_of_keywords = ["template_", "enum_", "Table", "BreakdownOfEnergyConsumptionAxis", "Hypercube"]

for kw in list_of_keywords:
    df1 = df1[df1["name_ranges"].str.contains(kw) == False]
df1 = df1.reset_index(drop=True)
df1

,name_ranges,sheets,cell_ranges,cell_shapes,cell_values
0,AddressOfSite,General Information,$D$450:$D$549,"[4, 450, 4, 549]","[[Duomo Square], [None], [None], [None], [None..."
1,AmountOfEmissionToAir,Environmental Disclosures,$G$80:$I$179,"[7, 80, 9, 179]","[[567, None, None], [None, None, None], [None,..."
2,AmountOfEmissionToAir_unit,Environmental Disclosures,$C$78,"[3, 78]",[[metric tonnes (t)]]
3,AmountOfEmissionToSoil,Environmental Disclosures,$K$80:$K$179,"[11, 80, 11, 179]","[[None], [67], [None], [None], [None], [None],..."
4,AmountOfEmissionToSoil_unit,Environmental Disclosures,$C$78,"[3, 78]",[[metric tonnes (t)]]
...,...,...,...,...,...
190,WasteDivertedToRecycleOrReuseVolume,Environmental Disclosures,$J$323:$J$422,"[10, 323, 10, 422]","[[1200], [800], [500], [300], [100], [4000], [..."
191,WasteDivertedToRecycleOrReuseVolume_unit,Environmental Disclosures,$G$323:$I$422,"[7, 323, 9, 422]","[[metric tonnes (t), None, None], [kilograms (..."
192,WaterDischargeFromUndertakingProductionProcesses,Environmental Disclosures,$D$310,"[4, 310]",[[100]]
193,WeightOfMaterialUsed,Environmental Disclosures,$G$435:$I$534,"[7, 435, 9, 534]","[[1500, None, None], [800, None, None], [None,..."


In [3]:
df1 = pd.read_pickle("df1.pkl")
df1

,name_ranges,sheets,cell_ranges,cell_shapes,cell_values
0,AddressOfSite,General Information,$D$450:$D$549,"[4, 450, 4, 549]","[[Duomo Square], [None], [None], [None], [None..."
1,AmountOfEmissionToAir,Environmental Disclosures,$G$80:$I$179,"[7, 80, 9, 179]","[[567, None, None], [None, None, None], [None,..."
2,AmountOfEmissionToAir_unit,Environmental Disclosures,$C$78,"[3, 78]",[[metric tonnes (t)]]
3,AmountOfEmissionToSoil,Environmental Disclosures,$K$80:$K$179,"[11, 80, 11, 179]","[[None], [67], [None], [None], [None], [None],..."
4,AmountOfEmissionToSoil_unit,Environmental Disclosures,$C$78,"[3, 78]",[[metric tonnes (t)]]
...,...,...,...,...,...
192,WasteDivertedToRecycleOrReuseVolume,Environmental Disclosures,$J$323:$J$422,"[10, 323, 10, 422]","[[1200], [800], [500], [300], [100], [4000], [..."
193,WasteDivertedToRecycleOrReuseVolume_unit,Environmental Disclosures,$G$323:$I$422,"[7, 323, 9, 422]","[[metric tonnes (t), None, None], [kilograms (..."
194,WaterDischargeFromUndertakingProductionProcesses,Environmental Disclosures,$D$310,"[4, 310]",[[100]]
195,WeightOfMaterialUsed,Environmental Disclosures,$G$435:$I$534,"[7, 435, 9, 534]","[[1500, None, None], [800, None, None], [None,..."


In [18]:
df[df["name_ranges"].str.contains("Table") == True]

,name_ranges,sheets,cell_ranges,cell_shapes,cell_values
1,AmountOfEmissionsTable,Environmental Disclosures,$D$80:$K$179,"[4, 80, 11, 179]","[[Alachlor, None, None, 567, None, None, None,..."
21,BreakdownOfEnergyConsumptionTable,Environmental Disclosures,$G$12:$K$14,"[7, 12, 11, 14]",None
120,ListOfSitesTable,General Information,$C$450:$H$549,"[3, 450, 8, 549]",None
121,ListOfSubsidiaryTable,General Information,$C$336:$E$435,"[3, 336, 5, 435]","[[1, Company XYZ Subsidiary 1, Street XYZ, Cit..."
127,NumberOfEmployeesForCountryOfEmploymentContrac...,Social Disclosures,$C$29:$H$128,"[3, 29, 8, 128]","[[Italy, None, 140, None, None, None], [Belgiu..."
155,SitesInBiodiversitySensitiveAreasTable,Environmental Disclosures,$C$188:$K$287,"[3, 188, 11, 287]",None


In [ ]:
df["cell_values"].iloc[0].append([None] for)
df["cell_values"].iloc[0]

In [44]:
df[df["name_ranges"] == "RateOfRecordableWorkRelatedAccidentsInTheReportingPeriod"]

,name_ranges,sheets,cell_ranges,cell_shapes,cell_values
142,RateOfRecordableWorkRelatedAccidentsInTheRepor...,Social Disclosures,$D$143,"[4, 143]",None


In [4]:
df = pd.read_pickle("df.pkl")
df

,name_ranges,sheets,cell_ranges,cell_shapes,cell_values
0,AddressOfSite,General Information,$D$450:$D$549,"[4, 450, 4, 549]","[[Duomo Square], [None], [None], [None], [None..."
1,AmountOfEmissionsTable,Environmental Disclosures,$D$80:$K$179,"[4, 80, 11, 179]","[[Alachlor, None, None, 567, None, None, None,..."
2,AmountOfEmissionToAir,Environmental Disclosures,$G$80:$I$179,"[7, 80, 9, 179]","[[567, None, None], [None, None, None], [None,..."
3,AmountOfEmissionToAir_unit,Environmental Disclosures,$C$78,"[3, 78]",[[metric tonnes (t)]]
4,AmountOfEmissionToSoil,Environmental Disclosures,$K$80:$K$179,"[11, 80, 11, 179]","[[None], [67], [None], [None], [None], [None],..."
...,...,...,...,...,...
291,WasteDivertedToRecycleOrReuseVolume,Environmental Disclosures,$J$323:$J$422,"[10, 323, 10, 422]","[[1200], [800], [500], [300], [100], [4000], [..."
292,WasteDivertedToRecycleOrReuseVolume_unit,Environmental Disclosures,$G$323:$I$422,"[7, 323, 9, 422]","[[metric tonnes (t), None, None], [kilograms (..."
293,WaterDischargeFromUndertakingProductionProcesses,Environmental Disclosures,$D$310,"[4, 310]",[[100]]
294,WeightOfMaterialUsed,Environmental Disclosures,$G$435:$I$534,"[7, 435, 9, 534]","[[1500, None, None], [800, None, None], [None,..."
